In [ ]:
%matplotlib inline
import matplotlib
matplotlib.rc("font", size=18)
import matplotlib.pyplot as plt
import yt
import numpy as np
from yt.units import kboltz, mp

In [ ]:
X_H = 0.76 # hydrogen mass fraction

Later we will make a cut on particles with kT > 30 eV, so figure out what the corresponding temperature in Kelvin is:

In [ ]:
kT_low = yt.YTQuantity(0.03, "keV") # 30 eV
T_low = float(kT_low.to_equivalent("K", "thermal"))

Load up the dataset:

In [ ]:
fn = "my_filename"
ds = yt.load(fn)

Here I'm assuming there's no electron number density field, so I assume full ionization to calculate $\mu_e$ and create an electron density field:

In [ ]:
mue = 4.0 / (3.0 * X_H + 1.0 + 4.0 * X_H)
def _n_e(field, data):
    return data["gas", "density"]/(mp*mue)
ds.add_field(("gas", "n_e"), function=_n_e, units="cm**-3")

Now that I have an electron density field I can set up an entropy field:

In [ ]:
def _entropy(field, data):
    return (kboltz*data["gas", "temperature"]).to("keV")*data["gas", "n_e"]**(-2./3.)
ds.add_field(("gas", "entropy"), function=_entropy, units="keV*cm**2", force_override=True)

Create a sphere, you can change the center and radius parameters as needed:

In [ ]:
center = "max" # or something
radius = (1.0, "Mpc") # or something
sp = ds.sphere(center, radius)

and now we can make a cut region where we cut out the particles with kT < 30 eV:

In [ ]:
cr = sp.cut_region(["obj['gas', 'temperature'] > %s" % T_low])

These are the fields I want to profile:

In [ ]:
fields = [("gas", "density"), 
          ("gas", "temperature"), 
          ("gas", "metallicity"), 
          ("gas", "n_e"),
          ("gas", "entropy")]

and I'm going to set the radial extrema for each particle type and the units of various things:

In [ ]:
gas_extrema = {"radius": (2.0, 1000.0)}
gas_units = {"radius": "kpc"}
particle_extrema = {("io", "particle_radius"): (2.0, 1000.0)}
particle_units = {("io", "particle_radius"): "kpc", 
                  ("io", "particle_mass"): "Msun"}

I will make two profiles, one from the sphere itself and another from the part of the sphere that only has hot gas with kT > 30 eV:

In [ ]:
ps = sp.profile("radius", fields, extrema=gas_extrema,
                logs={"radius": True}, units=gas_units, n_bins=60, 
                weight_field='ones')
pc = cr.profile("radius", fields, extrema=gas_extrema,
                logs={"radius": True}, units=gas_units, n_bins=60, 
                weight_field='ones')

Electron density plot:

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111)
ax.plot(ps.x, ps["n_e"], label='All Particles')
ax.plot(pc.x, pc["n_e"], label='Particles w/ kT > 30 eV')
ax.set_xscale("log")
ax.set_yscale("log")
ax.set_xlabel("r (kpc)")
ax.set_ylabel("$\mathrm{n_e\ (cm^{-3})}$")
ax.legend()
fig.savefig("halo_density.png")

Temperature plot:

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111)
ax.plot(ps.x.to("kpc"), (kboltz*ps["temperature"]).to("keV"), label='All Particles')
ax.plot(pc.x.to("kpc"), (kboltz*pc["temperature"]).to("keV"), label='Particles w/ kT > 30 eV')
ax.set_xscale("log")
ax.set_xlabel("r (kpc)")
ax.set_ylabel("T (keV)")
ax.legend()
fig.savefig("halo_kT.png")

Metallicity plot:

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111)
ax.plot(ps.x.to("kpc"), ps["metallicity"].to("Zsun"), label='All Particles')
ax.plot(pc.x.to("kpc"), pc["metallicity"].to("Zsun"), label='Particles w/ kT > 30 eV')
ax.set_xscale('log')
ax.set_xlabel("r (kpc)")
ax.set_ylabel("$\mathrm{Z\ (Z_\odot)}$")
ax.legend()
fig.savefig("halo_Z.png")

Entropy plot:

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111)
ax.plot(ps.x.to("kpc"), ps["entropy"], label='All Particles')
ax.plot(pc.x.to("kpc"), pc["entropy"], label='Particles w/ kT > 30 eV')
ax.set_xscale('log')
ax.set_xlabel("r (kpc)")
ax.set_ylabel("$\mathrm{S\ (keV\ cm^2)}$")
ax.legend()
fig.savefig("halo_entropy.png")

I now want to set the units of the gas particle mass to $M_\odot$:

In [ ]:
gas_units["gas", "cell_mass"] = "Msun"

And I will now make accumulated mass profiles of gas, dark matter, and stars:

In [ ]:
# DM+Stars
pmp = sp.profile(("io", "particle_radius"), [("io", "particle_mass")], 
                 extrema=particle_extrema, n_bins=128, weight_field=None, accumulation=True,
                 units=particle_units)
# Gas
pmg = sp.profile("radius", [("gas", "cell_mass")], 
                 extrema=gas_extrema, n_bins=128, weight_field=None, accumulation=True,
                 units=gas_units)

And now I can make the mass plot:

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111)
ax.plot(pmp.x, pmp["particle_mass"]+pmg["cell_mass"], label='Total Mass')
ax.plot(pmg.x, pmg["cell_mass"], label='Gas Mass')
ax.set_xscale('log')
ax.set_yscale("log")
ax.set_xlabel("r (kpc)")
ax.set_ylabel("$\mathrm{M\ (M_\odot)}$")
ax.legend()
fig.savefig("halo_masses.png")